In [111]:
import os
import zipfile
import fitz  # PyMuPDF
import re
import math
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import shutil
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

In [112]:
# Solo una vez al inicio
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()  # No entrenamiento

C:\Users\Jose\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\Jose\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jose\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an adminis

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [114]:
def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        return math.exp(loss.item())

In [99]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

In [100]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.lower()

In [101]:
def entropy(text):
    probs = [freq / len(text) for freq in Counter(text).values()]
    return -sum(p * math.log2(p) for p in probs)

In [102]:

def ngram_metrics(text, n=3):
    vectorizer = CountVectorizer(ngram_range=(n, n))
    ngrams = vectorizer.fit_transform([text])
    return dict(zip(vectorizer.get_feature_names_out(), ngrams.toarray()[0]))

In [103]:
def burstiness(text):
    words = text.split()
    word_counts = Counter(words)
    freqs = list(word_counts.values())
    if len(freqs) < 2:
        return 0.0
    return np.std(freqs) / np.mean(freqs)

In [104]:
def top_ngrams(text, n=3, top_k=10):
    vectorizer = CountVectorizer(ngram_range=(n, n))
    ngrams = vectorizer.fit_transform([text])
    freqs = ngrams.toarray()[0]
    ngram_names = vectorizer.get_feature_names_out()
    sorted_ngrams = sorted(zip(ngram_names, freqs), key=lambda x: x[1], reverse=True)
    return sorted_ngrams[:top_k]

In [105]:
def extract_zip(zip_path, extract_to="pdfs_temp"):
    os.makedirs(extract_to, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    pdf_files = []
    for root, dirs, files in os.walk(extract_to):
        for f in files:
            if f.endswith(".pdf"):
                pdf_files.append(os.path.join(root, f))
    
    return pdf_files

In [115]:
def process_pdfs_in_zip(zip_path):
    pdf_paths = extract_zip(zip_path)
    
    results = []

    for path in pdf_paths:
        print(f"Procesando: {os.path.basename(path)}")
        raw = extract_text_from_pdf(path)
        clean = preprocess_text(raw)
        ent = entropy(clean)
        bur = burstiness(clean)
        ppl = calculate_perplexity(clean)

        results.append({
            "archivo": os.path.basename(path),
            "entropia": round(ent, 4),
            "burstiness": round(bur, 4),
            "perplejidad": round(ppl, 4)
        })

    return pd.DataFrame(results)

In [116]:
def run_pipeline(zip_path, output_csv="resultados.csv"):
    df = process_pdfs_in_zip(zip_path)
    
    shutil.rmtree("pdfs_temp", ignore_errors=True)
    print("🧹 Carpeta 'pdfs_temp' eliminada.")
    
    return df

In [117]:
df_resultados = run_pipeline("Documents.zip")

display(df_resultados)


Procesando: D1.pdf
Procesando: D2.pdf
Procesando: D3.pdf
Procesando: D4.pdf
Procesando: D5.pdf
Procesando: D6.pdf
🧹 Carpeta 'pdfs_temp' eliminada.


,archivo,entropia,burstiness,perplejidad
0,D1.pdf,4.0744,0.5679,160.4222
1,D2.pdf,4.0528,0.5415,108.0962
2,D3.pdf,3.9881,0.6576,219.6888
3,D4.pdf,4.0383,0.6880,110.8801
4,D5.pdf,4.1078,0.7635,173.1369
5,D6.pdf,4.0451,0.4651,196.9493
